# Step 1: Environment Initialization
This cell installs the core engine components:
* **LangChain**: The framework orchestrating our AI agents.
* **OpenAI/Tenacity**: Reliable connectivity and exponential backoff retry logic.
* **HTTPX**: High-performance asynchronous networking.

In [ ]:
%pip install -U langchain langchain-community langchain-classic openai tenacity httpx

# Step 2: DeepSeek Model Connectivity
Loads the specialized **langchain-deepseek** module to allow our agents to access the DeepSeek V3 (logic) and R1 (reasoning) models.

In [ ]:
%pip install -U langchain-deepseek

# Step 3: API & Web Server Setup
Installs **FastAPI** and **Uvicorn** to host the interactive dashboard, along with **nest_asyncio** to allow the server to run directly inside this Jupyter Notebook.

In [ ]:
%pip install fastapi uvicorn nest_asyncio

# Step 4: Market Data Connectors
Installs **yahooquery** for real-time institutional-grade data fetching and **pandas** for advanced data frame manipulation.

In [ ]:
%pip install yahooquery pandas

# Step 5: System Imports
Importing foundational Python libraries for data cleaning (re), mathematical volatility calculations (math), and secure credential handling (getpass).

In [ ]:
import os # 'os' lets python talk to the computer's operating system to get hidden passwords
import re # 're' stands for Regular Expressions, used to find patterns in text (like cleaning AI  output)
import math # 'math' gives us advanced mathematical functions like sqrt or volatility
from datetime import date, datetime, timedelta 
# date.today()           → today's date as YYYY-MM-DD
# timedelta(days=90)     → lets us compute "90 days ago" for filtering old trades
# datetime.strptime()    → converts a string like "2024-11-15" into a date object
from getpass import getpass

# Step 6: LangChain Agent Architecture
Importing the components that define our AI's "brain": **PromptTemplates** for instructions and **OutputParsers** to ensure the AI speaks in a format our dashboard understands.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import LLMChain, SequentialChain 
from langchain_deepseek import ChatDeepSeek

print("✅ Imports OK")

# Step 7: API Key Management
Securely prompts for your DeepSeek API key. This key is stored in the local environment memory and is never saved in the notebook file for your security.

In [ ]:
import os
from getpass import getpass

# ── Deepseek key ────────────────────────────────────────────────────
DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY") 
if not DEEPSEEK_API_KEY:
    DEEPSEEK_API_KEY = getpass("🔑 Enter your DeepSeek API Key: ")
    
os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY # <-- Forces it into the environment

_DEEPSEEK_LIVE = len(DEEPSEEK_API_KEY) > 20
print(f"DeepSeek key:    {'✅ looks valid (' + str(len(DEEPSEEK_API_KEY)) + ' chars)' if _DEEPSEEK_LIVE else '❌ TOO SHORT — check key'}")

# Step 8: Advanced Data Analytics Engine
This is the heart of our data pipeline. It performs three critical tasks:
1. **Ticker Resolution**: Converts names (e.g., "Apple") into market symbols.
2. **YF Data Fetching**: Pulls 5 years of history to calculate **Drift (mu)** and **Volatility (sigma)**.
3. **Macro Caching**: Tracks the VIX, 10Y Yields, and live **USD/CAD Forex rates**.

In [ ]:
from yahooquery import Ticker, search
import pandas as pd
import numpy as np
import json
import time

_macro_cache = {"data": None, "ts": 0}

def resolve_ticker_sync(query: str) -> str:
    query = query.strip()
    try:
        res = search(query)
        if 'quotes' in res and len(res['quotes']) > 0:
            for q in res['quotes']:
                if q.get('symbol', '').upper() == query.upper(): return q['symbol'].upper()
            for q in res['quotes']:
                ex = q.get('exchange', '').upper()
                if ex in ['TOR', 'VAN', 'CNQ']: return q['symbol'].upper()
            return res['quotes'][0]['symbol'].upper()
    except Exception: pass
    query = query.upper()
    return query if "." in query else query + ".TO"

def get_yf_data(ticker_symbol: str, horizon: str = "medium", retries=3) -> str:
    h_map = {"short": 1.0, "medium": 5.0, "long": 10.0}
    years = h_map.get(horizon, 5.0)
    
    for attempt in range(retries):
        try:
            tk = Ticker(ticker_symbol)
            hist = tk.history(period="5y")
            if not isinstance(hist, pd.DataFrame) or hist.empty:
                if attempt < retries - 1:
                    time.sleep(1.5)
                    continue
                return json.dumps({"Error": f"Ticker {ticker_symbol} not found."})

            if isinstance(hist.index, pd.MultiIndex):
                try: hist = hist.xs(ticker_symbol, level='symbol')
                except: pass

            close_col = 'close' if 'close' in hist.columns else 'Close'
            ma_50 = hist[close_col].rolling(window=50).mean().iloc[-1]
            ma_200 = hist[close_col].rolling(window=200).mean().iloc[-1]
            current_price = float(hist[close_col].iloc[-1])
            
            # --- CALCULATE STATS FOR MONTE CARLO ---
            returns = hist[close_col].pct_change().dropna()
            mu = returns.mean() * 252 
            sigma = returns.std() * np.sqrt(252)

            # --- NEW: FUNDAMENTAL DATA & NEWS ---
            summary = tk.summary_detail.get(ticker_symbol, {})
            profile = tk.summary_profile.get(ticker_symbol, {})
            fin_data = tk.financial_data.get(ticker_symbol, {})
            
            # Extract Dividend and Currency safely
            currency = summary.get("currency", "CAD") if isinstance(summary, dict) else "CAD"
            div_yield = summary.get("dividendYield", 0.0) if isinstance(summary, dict) else 0.0
            div_yield = div_yield if not pd.isna(div_yield) else 0.0
            
            # Extract Live News Headlines
            try:
                news_data = tk.news
                headlines = [n['title'] for n in news_data[:3]] if isinstance(news_data, list) else []
            except: headlines = []
            
            data = {
                "Ticker": ticker_symbol,
                "Company Sector": profile.get('sector', 'Unknown') if isinstance(profile, dict) else 'Unknown',
                "Industry": profile.get('industry', 'Unknown') if isinstance(profile, dict) else 'Unknown',
                "Native Currency": currency.upper() if isinstance(currency, str) else "CAD",
                "Dividend Yield": f"{div_yield*100:.2f}%",
                "Recent News": " | ".join(headlines) if headlines else "No recent news available.",
                "Current Price": current_price,
                "Annual Expected Return (mu)": mu,
                "Annual Volatility (sigma)": sigma,
                "Market Cap": summary.get("marketCap", "N/A") if isinstance(summary, dict) else "N/A",
                "50-Day MA": float(round(ma_50, 2)) if not pd.isna(ma_50) else "N/A",
                "200-Day MA": float(round(ma_200, 2)) if not pd.isna(ma_200) else "N/A",
                "Trailing P/E": summary.get("trailingPE", "N/A") if isinstance(summary, dict) else "N/A"
            }
            return json.dumps(data, indent=2)
        except Exception as e:
            if attempt < retries - 1: time.sleep(1.5); continue
            return json.dumps({"Error": str(e), "mu": 0.08, "sigma": 0.15})

def get_macro_data_cached(ttl=300):
    global _macro_cache
    if _macro_cache["data"] and (time.time() - _macro_cache["ts"]) < ttl:
        return _macro_cache["data"]
    try:
        # Added CAD=X to track live USD to CAD Forex Rate
        symbols = {"^VIX": "VIX", "^TNX": "US_10Y", "^IRX": "US_3M", "CAD=X": "USD_CAD"}
        data = {}
        for sym, name in symbols.items():
            h = Ticker(sym).history(period="5d")
            col = 'close' if 'close' in h.columns else 'Close'
            data[name] = round(float(h[col].iloc[-1]), 4) if not h.empty else "N/A"
        _macro_cache = {"data": json.dumps(data, indent=2), "ts": time.time()}
        return _macro_cache["data"]
    except Exception: return json.dumps({"Error": "Macro unavailable"})

print("✅ Advanced Data Engine Ready.")

# Step 9: Data Integrity & Pydantic Validation
Defines strict schemas to ensure the AI agents provide valid data. This prevents the dashboard from crashing if an agent provides an unexpected response format.

In [ ]:
from pydantic import BaseModel, Field, field_validator
import re
from typing import Literal

class MarketAnalysisOutput(BaseModel):
    current_price: str = Field(alias="Current Price")
    valuation_cap: str = Field(alias="Valuation & Cap")
    trend_analysis: str = Field(alias="Trend Analysis")
    momentum_volume: str = Field(alias="Momentum & Volume")
    signal: Literal["BULLISH", "BEARISH", "NEUTRAL"] = Field(alias="Signal")
    confidence_score: str = Field(alias="Confidence Score")

    @field_validator("confidence_score", mode="before")
    @classmethod
    def validate_confidence(cls, v):
        match = re.search(r"(\d{1,3})%", v)
        if not match or not (0 <= int(match.group(1)) <= 100):
            raise ValueError(f"Confidence must be 0-100%, got: {v}")
        return v

def parse_kv_output(raw: str, model_cls=None):
    """Parses raw KV text into dictionary and optionally tests Pydantic validation."""
    lines = raw.strip().split("\n")
    data = {}
    for line in lines:
        if ":" in line:
            key, _, value = line.partition(":")
            data[key.strip()] = value.strip()
    
    if model_cls:
        try:
            model_cls(**data)
        except Exception as e:
            print(f"Validation Warning: {e}")
            
    return data

print("✅ Pydantic Validation schemas loaded.")

# Step 10: Institutional Core ETF Universe
Defines our "Safe Haven" assets. We use a **Risk-Parity** model where safer assets (like Bonds) get higher weights than volatile ones (like Energy) to ensure a stable portfolio baseline.

In [ ]:
CORE_ETFS = {
    # "TICKER.TO": ("human-readable description",  annual_volatility_as_decimal)
    #                                               ↑ lower = safer = gets bigger weight

    "XIU.TO" : ("Canadian Large-Cap (S&P/TSX 60)",         0.13),
    # XIU holds the 60 biggest TSX companies. Think: RBC, Shopify, Suncor.
    # 13% annual vol = moderate. Your $10k could swing ±$1,300 in a bad year.

    "ZAG.TO" : ("Canadian Bonds — stability anchor",        0.04),
    # Government + corporate bonds. Boring but stable. Rises when stocks crash.
    # 4% vol = lowest in our universe → gets the MOST weight in risk-parity.

    "HUG.TO" : ("Gold — inflation hedge (priced in CAD)",   0.12),
    # Gold doesn't correlate with stocks or bonds — it's "crisis insurance".
    # HUG.TO is CAD-denominated so you don't get double-hit by a strong USD.

    "XRE.TO" : ("Canadian REITs — income",                  0.17),
    # Real estate investment trusts. High dividends but sensitive to interest rates.
    # When BoC hikes, REITs fall (borrowing costs up). When BoC cuts, REITs rise.

    "XEG.TO" : ("Canadian Energy — oil sands",              0.22),
    # Suncor, CNQ, Cenovus etc. Energy is ~17% of the TSX.
    # 22% vol = highest in core → gets LEAST weight in risk-parity.

    "XEF.TO" : ("International Developed Markets (EAFE)",   0.14),
    # Europe, Australia, Japan, etc. Diversifies away from Canada's resource bias.

    "VFV.TO" : ("U.S. S&P 500 — unhedged USD exposure",     0.16),
    # The 500 biggest US companies. "Unhedged" means if CAD weakens,
    # your VFV position becomes worth MORE in CAD terms (built-in hedge).

    "ZEB.TO" : ("Canadian Big-Six Banks, equal-weight",     0.16),
    # RBC, TD, BMO, BNS, CM, NA in equal portions.
    # Banks are 20%+ of TSX. Equal-weight reduces single-bank risk vs XIU.

    "XIT.TO" : ("Canadian Tech + Defence adjacency",        0.20),
    # Shopify dominates but also includes CAE (flight simulators / defence).
}

# ── RISK-PARITY WEIGHTS ──────────────────────────────────────────────
# FORMULA: w_i = (1/σ_i) / Σ(1/σ_j)
# INTUITION: ZAG has σ=4% → inverse = 1/0.04 = 25
#            XEG has σ=22% → inverse = 1/0.22 ≈ 4.5
#            ZAG gets 25/(25+4.5+...) of the portfolio — much larger slice.
# RESULT: Each ETF contributes EQUAL RISK dollars, not equal dollar amounts.
#         This means no single ETF can blow up the portfolio.
_inv_vols = {t: 1.0 / vol for t, (_, vol) in CORE_ETFS.items()}
_inv_sum  = sum(_inv_vols.values())  # denominator of the formula
RP_WEIGHTS = {t: round(inv / _inv_sum, 4) for t, inv in _inv_vols.items()}

# Print a visual bar chart of the weights so you can verify intuitively
print("Risk-parity baseline weights (longer bar = more weight = safer ETF):")
for t, w in RP_WEIGHTS.items():
    bar = "█" * int(w * 200)  # scale bars to readable length
    desc = CORE_ETFS[t][0][:35]  # truncate long descriptions
    print(f"  {t:<10} {w*100:5.1f}%  {bar}  {desc}")

print(f"\n  Sum of weights: {sum(RP_WEIGHTS.values())*100:.1f}%  ← should be ~100%")

# Step 11: Allocation & Horizon Logic
Configures the "Satellite" exploration budget. This determines how much "play money" the AI can use for your target stocks based on your time horizon and maximum drawdown limits.

In [ ]:
# Portfolio Configuration Constants
HORIZON_CONFIG = {
    "short": {"label": "< 2 years", "max_exp": 0.10},
    "medium": {"label": "2–7 years", "max_exp": 0.15},
    "long": {"label": "> 7 years", "max_exp": 0.20},
}

CORE_ETFS = {
    "XIU.TO": ("Canadian Large-Cap", 0.13),
    "ZAG.TO": ("Canadian Bonds", 0.04),
    "HUG.TO": ("Gold (CAD)", 0.12),
    "XRE.TO": ("Canadian REITs", 0.17),
    "XEG.TO": ("Canadian Energy", 0.22),
    "XEF.TO": ("Intl Developed", 0.14),
    "VFV.TO": ("U.S. S&P 500", 0.16),
    "ZEB.TO": ("Canadian Banks", 0.16),
    "XIT.TO": ("Canadian Tech", 0.20),
}

_inv_vols = {t: 1.0 / vol for t, (_, vol) in CORE_ETFS.items()}
_inv_sum = sum(_inv_vols.values())
RP_WEIGHTS = {t: round(inv / _inv_sum, 4) for t, inv in _inv_vols.items()}

# Step 12: Dual-Model LLM Setup
Initializes two distinct AI personalities:
* **DeepSeek V3**: Fast and creative for technical and macro reports.
* **DeepSeek R1 (Reasoner)**: High-level mathematical reasoning for the Quant agent.

In [ ]:
# 1. DeepSeek V3 (Lightning fast, uncensored logic for text agents)
llm = ChatDeepSeek(
    model="deepseek-chat", 
    temperature=0.2,           
    api_key=DEEPSEEK_API_KEY,
    max_retries=3
)

# 2. DeepSeek R1 (Heavy reasoning model for the Quant agent)
quant_llm = ChatDeepSeek(
    model="deepseek-reasoner",
    api_key=DEEPSEEK_API_KEY,
    max_retries=3
)

print("✅ DeepSeek V3 and DeepSeek R1 fully initialized.")

# Step 13: Multi-Agent Personalities
We define 6 specialized AI personas:
1. **Market Analyst**: Technicals & News Sentiment.
2. **Macro Manager**: Global Regime Detection.
3. **Quant Scientist**: Statistical Edge & Kelly Sizing.
4. **CIO Strategist**: Core-Satellite Portfolio Architecture.
5. **Risk Officer**: Taleb-inspired Veto Authority.
6. **Executive Advisor**: Final Wealthsimple Trade Instructions.

# AGENT 1: MARKET ANALYST

In [ ]:
# 1. MARKET ANALYST (John J. Murphy)
market_prompt = PromptTemplate(
    input_variables=["ticker", "yf_data"],
    template="""SYSTEM: ROLE: Elite Technical & Fundamental Analyst. TARGET: {ticker}
RAW DATA: {yf_data}

INSTRUCTIONS: 
1. Open <thinking> tags. Evaluate Moving Averages, P/E ratio, Sector positioning, and the sentiment of the Live News headlines.
2. Close </thinking> tags and output highly detailed Key: Value pairs.

REQUIRED FORMAT:
<thinking>[Evaluate technicals + news sentiment]</thinking>
Sector & Context: [State the Sector and summarize the impact of recent news headlines]
Currency & Yield: [State the Native Currency and Dividend Yield]
Current Price: [Value]
Valuation Profile: [Cap size and PE analysis]
Trend Analysis: [Detail the 50/200 MA interaction]
Signal: [BULLISH/BEARISH/NEUTRAL]
Confidence Score: [0% to 100%]

CRITICAL: Do NOT use markdown (*, #). Keep values to one single line."""
)
market_runnable = market_prompt | llm | StrOutputParser()

print("✅ Market agent ready")

# AGENT 2: MACRO ANALYST

In [ ]:
# 2. MACRO ANALYST (Ernest Chan)
macro_prompt = PromptTemplate(
    input_variables=["ticker", "horizon", "macro_data"],
    template="""SYSTEM: ROLE: Global Macro Portfolio Manager executing Ernest Chan's regime detection.
TARGET ASSET: {ticker} | HORIZON: {horizon}
MACRO DATA: {macro_data}

INSTRUCTIONS: 
1. Open <thinking> tags to deduce the current global economic regime using VIX (volatility) and Yield Curves (US_10Y, US_3M).
2. Close </thinking> tags and output a structured macroeconomic impact report.

REQUIRED FORMAT:
<thinking>[Macroeconomic regime deliberation]</thinking>
Regime Observation: [RISK-ON/OFF/TRANSITIONAL]
Volatility Assessment: [Analyze VIX implications for this specific asset]
Yield Curve Impact: [Analyze short vs long term rates]
Macro Risk Score: [0% to 100% - Higher means more dangerous]

CRITICAL: Do NOT use markdown (*, #). Use plain text ONLY."""
)
macro_runnable = macro_prompt | llm | StrOutputParser()

print("✅ Macro agent ready")

# AGENT 3:  QUANT ANALYST

In [ ]:
# 3. QUANT ANALYST (Marcos López de Prado)
quant_prompt = PromptTemplate(
    input_variables=["ticker", "market_analysis", "macro_analysis"],
    template="""SYSTEM: ROLE: Systematic Quant executing Marcos Lopez de Prado's advanced financial machine learning logic.
TARGET ASSET: {ticker}
MARKET INPUT: {market_analysis}
MACRO INPUT: {macro_analysis}

INSTRUCTIONS: 
1. Open <thinking> tags. Compute statistical edge, Kelly criterion sizing, and evaluate meta-labeling probabilities based on the provided inputs.
2. Close </thinking> tags and output a strict quantitative synthesis.

REQUIRED FORMAT:
<thinking>[Rigorous statistical and probabilistic reasoning]</thinking>
Statistical Edge: [Describe the mathematical probability of success]
Kelly Logic: [Explain optimal sizing logic]
Target Allocation: [0% to 100%]
Model Conviction: [0% to 100%]

CRITICAL: Do NOT use markdown (*, #). Use plain text ONLY."""
)
quant_runnable = quant_prompt | quant_llm | StrOutputParser()

print("✅ Quant agent ready")

# AGENT 4: CIO (Chief Investment Officer)

In [ ]:
# 4. CIO STRATEGY (François-Serge Lhabitant)
cio_prompt = PromptTemplate(
    input_variables=["ticker", "horizon", "budget_fmt", "core_cap_fmt", "explore_cap_fmt", "core_etfs", "market_analysis", "macro_analysis", "quant_analysis"],
    template="""SYSTEM: ROLE: Chief Investment Officer executing François-Serge Lhabitant's Core-Satellite construction.
MANDATE: Asset: {ticker} | Horizon: {horizon} | Total Budget: {budget_fmt} (Core: {core_cap_fmt}, Explore: {explore_cap_fmt})
CORE ETFS: {core_etfs}
RESEARCH: Market: {market_analysis} | Macro: {macro_analysis} | Quant: {quant_analysis}

INSTRUCTIONS: 
1. Open <thinking> tags. Synthesize all reports to create an optimized, institutional portfolio allocation.
2. Close </thinking> tags. Output the explicit percentage breakdown for the UI Donut Chart engine.

REQUIRED FORMAT:
<thinking>[Synthesis and allocation mathematics]</thinking>
Alpha Allocation: [BUY/AVOID/DEFER]
Strategic Rationale: [One concise sentence justifying the allocation]
Target Weight: [0% to 100%]
Core Construction: [List ETFs and exact % weights including the target ticker, e.g. XIU.TO: 40%, ZAG.TO: 40%, {ticker}: 20%]

CRITICAL: Do NOT use markdown (*, #). Use plain text ONLY."""
)
cio_runnable = cio_prompt | llm | StrOutputParser()

print("✅ CIO agent ready")

# AGENT 5: RISK MANAGER (VETO AUTHORITY)

In [ ]:
# 5. RISK MANAGER (Nassim Taleb)
risk_prompt = PromptTemplate(
    input_variables=["ticker", "cio_strategy", "mdd_pct", "budget_fmt", "budget", "market_analysis"],
    template="""SYSTEM: ROLE: Chief Risk Officer (Nassim Taleb).
TARGET ASSET: {ticker} | MAX DRAWDOWN LIMIT: {mdd_pct}
MARKET & SECTOR DATA: {market_analysis}
CIO STRATEGY PROPOSAL: {cio_strategy}

INSTRUCTIONS: 
1. Open <thinking> tags. Review the Sector & Context to detect correlation traps (e.g., too much Oil & Gas). Check if the CIO adhered to a safe Barbell strategy.
2. Close </thinking> tags. Issue a final veto or approval.

REQUIRED FORMAT:
<thinking>[Sector correlation check & Barbell validation]</thinking>
Sector Exposure Risk: [Evaluate industry concentration danger]
Barbell Compliance: [Check]
Max Drawdown Estimate: [0% to 100%]
Veto Status: [APPROVED/VETOED]

CRITICAL: Do NOT use markdown (*, #). Use plain text ONLY."""
)
risk_runnable = risk_prompt | llm | StrOutputParser()

print("✅ Risk manager ready")

## AGENT 6: EXECUTIVE SUMMARIZER 

In [ ]:
# 6. EXECUTIVE SUMMARIZER (Robert Kissell / Retail Advisor)
summary_prompt = PromptTemplate(
    input_variables=["tickers", "strategies", "budget_fmt", "core_etfs_with_prices", "mc_projections", "ticker_context", "usdcad_rate"],
    template="""SYSTEM: ROLE: Friendly, highly competent Retail Financial Advisor. 
TARGET AUDIENCE: Retail investors using Wealthsimple. 
STRATEGIES PRODUCED: {strategies}
MONTE CARLO DATA: {mc_projections}
ASSET CONTEXT (Currency & Yields): {ticker_context}
LIVE USD/CAD EXCHANGE RATE: {usdcad_rate}
BUDGET: {budget_fmt}
CORE PRICES: {core_etfs_with_prices}

INSTRUCTIONS: 
1. Open <thinking> tags. 
   - FOREX MATH: If an asset is in USD, multiply its price by {usdcad_rate} to get the CAD price BEFORE dividing the budget to calculate exact shares.
   - DIVIDEND MATH: Multiply the CAD Budget deployed into an asset by its Dividend Yield to estimate annual passive income.
   - BLEND: Calculate the Total Portfolio Expected Return.
2. Close </thinking> tags. Output the final action plan.

REQUIRED FORMAT:
<thinking>[Forex conversions, Share Math, Dividend Income Projections]</thinking>
Portfolio Stance: [BULLISH/BEARISH/NEUTRAL]
Action Required: [BUY/SELL/HOLD]
Capital Deployed: [0% to 100%]
Projected Annual Income: [$ Value CAD based on Dividend Yields]
Final Portfolio Breakdown: [e.g. XIU.TO: 60%, NVDA: 20%, Cash: 20%]
Expected Return: [Projected Portfolio Value & ROI % based on Monte Carlo]
Final Verdicts: [Plain English summary]
Wealthsimple Trade Orders: [Step-by-step instructions with EXACT share counts based on CAD budget]
Disclaimer: [Educational use only warning]

CRITICAL: Do NOT use markdown (*, #). Keep text flat and highly professional."""
)
summary_runnable = summary_prompt | llm | StrOutputParser()

print("✅ Summarizer ready")

# Step 14: Terminal Visual Styles
Defines the visual language of our reports, mapping internal AI signals to intuitive icons (🟢, 🟡, 🔴) for the manual fallback logs.

In [ ]:
VERDICT_STYLE = {
    "BUY_CORE" : ("🟢", "BUY  CORE "),  # very strong signal
    "BUY_SAT"  : ("🟢", "BUY  SAT.  "), # satellite/explore position
    "COND_BUY" : ("⚠️ ", "COND.BUY  "), # conditional — needs a trigger
    "DEFER"    : ("🟡", "DEFER      "), # wait for better setup
    "AVOID"    : ("🔴", "AVOID      "), # negative signal
}

def render(mandate, market_raw, macro_raw, quant_raw, cio_raw, risk_raw, ticker):
    print(f"\n{'═'*70}")
    print(f"  🍁 SAFETY-FIRST AI HEDGE FUND — Markdown Report")
    print(f"{'═'*70}")
    print(f"  📊 Ticker analysed : {ticker}")
    print(f"  💰 Budget          : {mandate['budget_fmt']}")
    print(f"  🕐 Horizon         : {mandate['horizon_label']}")
    print(f"  🛑 Max drawdown    : {mandate['mdd_pct']}")
    print(f"{'═'*70}\n")
    
    print("── 🌐 MACRO ANALYSIS (Agent 2) ──────────────────────────────────")
    print(macro_raw.strip() + "\n")
    
    print("── 📈 MARKET ANALYSIS (Agent 1) ─────────────────────────────────")
    print(market_raw.strip() + "\n")
    
    print("── 🧮 QUANTITATIVE EDGE (DeepSeek R1) ───────────────────────────")
    print(quant_raw.strip() + "\n")
    
    print("── 🧠 CIO STRATEGY (Agent 3) ────────────────────────────────────")
    print(cio_raw.strip() + "\n")
    
    print("── 🔒 RISK ASSESSMENT (Agent 4) ─────────────────────────────────")
    print(risk_raw.strip() + "\n")
    
    print(f"{'═'*70}")
    print("  ⚠️ EDUCATIONAL SIMULATION — NOT financial advice.")
    print(f"{'═'*70}\n")


# Step 15: Pipeline Resilience
A safety wrapper that forces the AI agents to retry up to 3 times if the API is busy or your internet connection flickers.

In [ ]:
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
import httpx
from openai import APIConnectionError

# ── RETRY LOGIC ──────────────────────────────────────────────────────
# WHY: If your wifi drops for 2 seconds, this catches the error and
#      waits exponentially (2s, 4s, 8s) before trying again, up to 3 times.
@retry(
    wait=wait_exponential(multiplier=1, min=2, max=10),
    stop=stop_after_attempt(3),
    retry=retry_if_exception_type((httpx.ConnectError, APIConnectionError, TimeoutError, OSError)),
    reraise=True
)
def safe_invoke(inputs: dict) -> dict:
    return pipeline.invoke(inputs)


# ── MAIN RUN FUNCTION ────────────────────────────────────────────────
def run(ticker: str, mandate: dict) -> dict: 
    print(f"\n🔄 Analysing {ticker.upper()}...")
    print("   Agents are gathering data and calculating edge — expect 60–120 seconds.\n")

    try:
        results = safe_invoke({
            "ticker"         : ticker.upper(),
            "horizon"        : mandate["horizon"],
            "budget_fmt"     : mandate["budget_fmt"],
            "budget"         : str(mandate["budget"]),
            "core_cap_fmt"   : mandate["core_cap_fmt"],
            "explore_cap_fmt": mandate["explore_cap_fmt"],
            "mdd_pct"        : mandate["mdd_pct"],
            "core_etfs"      : mandate["core_etfs"]
        })
    except Exception as e:
        print(f"❌ Network completely failed after multiple retries for {ticker}.")
        print(f"   Error details: {e}")
        return None 

    render(
        mandate    = mandate,
        market_raw = results["market_analysis"],
        macro_raw  = results["macro_analysis"],
        quant_raw  = results["quant_analysis"], # <-- ADDED
        cio_raw    = results["cio_strategy"],
        risk_raw   = results["risk_assessment"],
        ticker     = ticker,
    )
    
    return results

# Step 16: Launch HEDGE-FUND.AI Terminal
This cell boots the **FastAPI Server** and generates the **Icy Glassmorphism Dashboard**.
### Features:
* **Infinite Rotating Carousel**: Smoothly slide between multiple stock analyses.
* **Monte Carlo Simulator**: Visualizes the best, worst, and expected case for your money.
* **Forex Engine**: Automatically converts USD stocks to CAD share counts for Wealthsimple.
* **Dynamic Server**: Automatically kills old "Zombie" servers and finds a fresh port.

In [ ]:
import nest_asyncio, uvicorn, threading, socket, webbrowser, time, asyncio, queue, json, traceback, os, random
import numpy as np
import pandas as pd
from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse, StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek
from IPython.display import display, HTML

nest_asyncio.apply()
app = FastAPI()
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"])

class AnalyzeRequest(BaseModel):
    session_id: str; budget: float; horizon: str; mdd: float; tickers: str

progress_queues = {}

async def push_progress(sid, agent, status, ticker="", detail=""):
    if sid in progress_queues:
        await progress_queues[sid].put(json.dumps({"agent": agent, "status": status, "ticker": ticker, "detail": detail}))

# --- FAIL-SAFE CONSTANTS ---
HORIZON_CONFIG = {
    "short": {"label": "< 2 years", "max_exp": 0.10},
    "medium": {"label": "2–7 years", "max_exp": 0.15},
    "long": {"label": "> 7 years", "max_exp": 0.20},
}
CORE_ETFS = {
    "XIU.TO": ("Canadian Large-Cap", 0.13), "ZAG.TO": ("Canadian Bonds", 0.04),
    "HUG.TO": ("Gold (CAD)", 0.12), "XRE.TO": ("Canadian REITs", 0.17),
    "XEG.TO": ("Canadian Energy", 0.22), "XEF.TO": ("Intl Developed", 0.14),
    "VFV.TO": ("U.S. S&P 500", 0.16), "ZEB.TO": ("Canadian Banks", 0.16),
    "XIT.TO": ("Canadian Tech", 0.20),
}
_inv_vols = {t: 1.0 / vol for t, (_, vol) in CORE_ETFS.items()}
RP_WEIGHTS = {t: round(inv / sum(_inv_vols.values()), 4) for t, inv in _inv_vols.items()}

# --- SELF-CONTAINED DATA ENGINE ---
_safe_macro_cache = {"data": None, "ts": 0}

def get_macro_data_cached_safe(ttl=300):
    global _safe_macro_cache
    if _safe_macro_cache["data"] and (time.time() - _safe_macro_cache["ts"]) < ttl:
        return _safe_macro_cache["data"]
    try:
        from yahooquery import Ticker
        symbols = {"^VIX": "VIX", "^TNX": "US_10Y", "^IRX": "US_3M", "CAD=X": "USD_CAD"}
        data = {}
        for sym, name in symbols.items():
            tk = Ticker(sym)
            h = tk.history(period="5d")
            if isinstance(h, pd.DataFrame) and not h.empty:
                col = 'close' if 'close' in h.columns else 'Close'
                data[name] = round(float(h[col].iloc[-1]), 4)
            else:
                data[name] = "N/A"
        _safe_macro_cache = {"data": json.dumps(data, indent=2), "ts": time.time()}
        return _safe_macro_cache["data"]
    except Exception as e: return json.dumps({"Error": f"Macro unavailable: {str(e)}", "USD_CAD": 1.35})

def get_yf_data_safe(ticker_symbol: str, horizon: str = "medium", retries=2) -> str:
    for attempt in range(retries):
        try:
            from yahooquery import Ticker
            tk = Ticker(ticker_symbol)
            hist = tk.history(period="5y")
            if not isinstance(hist, pd.DataFrame) or hist.empty:
                if attempt < retries - 1: time.sleep(1.5); continue
                return json.dumps({"Error": f"Ticker {ticker_symbol} not found."})

            if isinstance(hist.index, pd.MultiIndex):
                try: hist = hist.xs(ticker_symbol, level='symbol')
                except: pass

            close_col = 'close' if 'close' in hist.columns else 'Close'
            ma_50 = hist[close_col].rolling(window=50).mean().iloc[-1]
            ma_200 = hist[close_col].rolling(window=200).mean().iloc[-1]
            current_price = float(hist[close_col].iloc[-1])
            
            returns = hist[close_col].pct_change().dropna()
            mu = float(returns.mean() * 252) if len(returns) > 0 else 0.08
            sigma = float(returns.std() * np.sqrt(252)) if len(returns) > 0 else 0.15

            summary_resp = tk.summary_detail
            summary = summary_resp.get(ticker_symbol, {}) if isinstance(summary_resp, dict) else {}
            
            profile_resp = tk.summary_profile
            profile = profile_resp.get(ticker_symbol, {}) if isinstance(profile_resp, dict) else {}
            
            currency = summary.get("currency", "CAD")
            div_yield = summary.get("dividendYield", 0.0)
            try: div_yield = float(div_yield) if div_yield is not None and not pd.isna(div_yield) else 0.0
            except (ValueError, TypeError): div_yield = 0.0
            
            try:
                news_data = tk.news
                headlines = [n['title'] for n in news_data[:3]] if isinstance(news_data, list) else []
            except: headlines = []
            
            data = {
                "Ticker": ticker_symbol,
                "Company Sector": profile.get('sector', 'Unknown'),
                "Industry": profile.get('industry', 'Unknown'),
                "Native Currency": currency.upper() if isinstance(currency, str) else "CAD",
                "Dividend Yield": f"{div_yield*100:.2f}%",
                "Recent News": " | ".join(headlines) if headlines else "No recent news available.",
                "Current Price": current_price,
                "Annual Expected Return (mu)": mu,
                "Annual Volatility (sigma)": sigma,
                "Market Cap": summary.get("marketCap", "N/A"),
                "50-Day MA": float(round(ma_50, 2)) if not pd.isna(ma_50) else "N/A",
                "200-Day MA": float(round(ma_200, 2)) if not pd.isna(ma_200) else "N/A",
                "Trailing P/E": summary.get("trailingPE", "N/A")
            }
            return json.dumps(data, indent=2)
        except Exception as e:
            if attempt < retries - 1: time.sleep(1.5); continue
            return json.dumps({"Error": str(e), "Annual Expected Return (mu)": 0.08, "Annual Volatility (sigma)": 0.15})

def resolve_ticker_sync(query: str) -> str:
    query = query.strip()
    try:
        from yahooquery import search
        res = search(query)
        if 'quotes' in res and len(res['quotes']) > 0:
            for q in res['quotes']:
                if q.get('symbol', '').upper() == query.upper(): return q['symbol'].upper()
            for q in res['quotes']:
                ex = q.get('exchange', '').upper()
                if ex in ['TOR', 'VAN', 'CNQ']: return q['symbol'].upper()
            return res['quotes'][0]['symbol'].upper()
    except Exception: pass
    query = query.upper()
    return query if "." in query else query + ".TO"

def get_core_prices_sync():
    try:
        from yahooquery import Ticker
        prices = Ticker(list(CORE_ETFS.keys())).price
        return "\n".join([f"{t}: ${round(prices[t].get('regularMarketPrice', 0), 2)}" if isinstance(prices, dict) and t in prices and isinstance(prices[t], dict) else f"{t}: N/A" for t in CORE_ETFS.keys()])
    except Exception: return "Prices currently unavailable."

# --- BULLETPROOF ASYNC EXECUTION ---
async def safe_ainvoke(runnable, inputs, retries=2):
    for attempt in range(retries):
        try:
            return await asyncio.wait_for(runnable.ainvoke(inputs), timeout=150.0)
        except Exception as e:
            if attempt == retries - 1: 
                return f"<thinking>API Error triggered fallback: {str(e)}</thinking>\nSignal: NEUTRAL\nConfidence Score: 0%\nTarget Allocation: 0%\nModel Conviction: 0%\nAlpha Allocation: DEFER\nVeto Status: VETOED\nAction Required: HOLD\nCapital Deployed: 0%\nExpected Return: N/A\nFinal Verdicts: API Timeout - Safety override engaged.\n"
            await asyncio.sleep(3)

@app.get("/progress/{session_id}")
async def progress_stream(session_id: str):
    if session_id not in progress_queues: progress_queues[session_id] = asyncio.Queue()
    async def event_generator():
        q = progress_queues[session_id]
        while True:
            try:
                msg = await asyncio.wait_for(q.get(), timeout=0.5)
                yield f"data: {msg}\n\n"
                if json.loads(msg).get("status") in ["complete", "error"]: break
            except asyncio.TimeoutError:
                yield f"data: {json.dumps({'agent':'heartbeat'})}\n\n"
    return StreamingResponse(event_generator(), media_type="text/event-stream")

@app.post("/analyze")
async def analyze_portfolio(req: AnalyzeRequest):
    sid = req.session_id
    if sid not in progress_queues: progress_queues[sid] = asyncio.Queue()
    try:
        ds_key = os.environ.get("DEEPSEEK_API_KEY", "").strip()
        if not ds_key or len(ds_key) < 10: 
            raise ValueError("DEEPSEEK_API_KEY is missing or invalid. Please run the API key cell above.")

        raw_inputs = [t.strip() for t in req.tickers.split(",") if t.strip()]
        if not raw_inputs: raw_inputs = ["XIU.TO"] 
        loop = asyncio.get_running_loop()
        
        safe_llm = ChatDeepSeek(model="deepseek-chat", temperature=0.2, api_key=ds_key, max_retries=2)
        safe_quant_llm = ChatDeepSeek(model="deepseek-reasoner", api_key=ds_key, max_retries=2)
        
        explore_frac = max(0.05, min(HORIZON_CONFIG[req.horizon]["max_exp"], (req.mdd/100 - 0.05) / 0.15 * 0.20))
        core_etfs_str = "\n".join(f"{t}: {desc} (RP: {RP_WEIGHTS[t]*100:.1f}%)" for t, (desc, _) in CORE_ETFS.items())

        mandate = {
            "budget": req.budget, "budget_fmt": f"${req.budget:,.2f} CAD", "horizon": req.horizon,
            "core_cap_fmt": f"${req.budget * (1-explore_frac):,.2f}", "explore_cap_fmt": f"${req.budget * explore_frac:,.2f}",
            "mdd_pct": f"{req.mdd}%", "core_etfs": core_etfs_str
        }

        market_runnable = PromptTemplate.from_template("SYSTEM: ROLE: Elite Technical & Fundamental Analyst. TARGET: {ticker}\nRAW DATA: {yf_data}\n\nINSTRUCTIONS: \n1. Open <thinking> tags. Evaluate MAs, P/E ratio, Sector positioning, and News sentiment.\n2. Close </thinking> tags and output strictly Key: Value pairs.\n\nREQUIRED FORMAT:\n<thinking>[Reasoning]</thinking>\nSector & Context: [Analysis]\nCurrency & Yield: [Value]\nCurrent Price: [Value]\nValuation Profile: [Analysis]\nTrend Analysis: [Analysis]\nSignal: [BULLISH/BEARISH/NEUTRAL]\nConfidence Score: [0% to 100%]\n\nCRITICAL: Do NOT use markdown.") | safe_llm | StrOutputParser()
        macro_runnable = PromptTemplate.from_template("SYSTEM: ROLE: Global Macro Portfolio Manager.\nTARGET ASSET: {ticker} | HORIZON: {horizon}\nMACRO DATA: {macro_data}\n\nINSTRUCTIONS: \n1. Open <thinking> tags to deduce the global economic regime.\n2. Close </thinking> tags and output Key: Value pairs.\n\nREQUIRED FORMAT:\n<thinking>[Reasoning]</thinking>\nRegime Observation: [RISK-ON/OFF/TRANSITIONAL]\nVolatility Assessment: [Analysis]\nYield Curve Impact: [Analysis]\nMacro Risk Score: [0% to 100%]\n\nCRITICAL: Do NOT use markdown.") | safe_llm | StrOutputParser()
        quant_runnable = PromptTemplate.from_template("SYSTEM: ROLE: Systematic Quant.\nTARGET ASSET: {ticker}\nMARKET: {market_analysis}\nMACRO: {macro_analysis}\n\nINSTRUCTIONS: \n1. Open <thinking> tags. Compute statistical edge and Kelly criterion.\n2. Close </thinking> tags and output Key: Value pairs.\n\nREQUIRED FORMAT:\n<thinking>[Reasoning]</thinking>\nStatistical Edge: [Analysis]\nKelly Logic: [Analysis]\nTarget Allocation: [0% to 100%]\nModel Conviction: [0% to 100%]\n\nCRITICAL: Do NOT use markdown.") | safe_quant_llm | StrOutputParser()
        cio_runnable = PromptTemplate.from_template("SYSTEM: ROLE: Chief Investment Officer.\nMANDATE: Asset: {ticker} | Horizon: {horizon} | Total Budget: {budget_fmt}\nCORE ETFS: {core_etfs}\nRESEARCH: Market: {market_analysis} | Macro: {macro_analysis} | Quant: {quant_analysis}\n\nINSTRUCTIONS: \n1. Open <thinking> tags. Synthesize all reports to create an optimized portfolio.\n2. Close </thinking> tags. Output explicit breakdown.\n\nREQUIRED FORMAT:\n<thinking>[Reasoning]</thinking>\nAlpha Allocation: [BUY/AVOID/DEFER]\nStrategic Rationale: [One sentence]\nTarget Weight: [0% to 100%]\nCore Construction: [List ETFs and exact % weights including {ticker}, e.g. XIU.TO: 40%, ZAG.TO: 40%, {ticker}: 20%]\n\nCRITICAL: Do NOT use markdown.") | safe_llm | StrOutputParser()
        risk_runnable = PromptTemplate.from_template("SYSTEM: ROLE: Chief Risk Officer.\nTARGET ASSET: {ticker} | MAX DRAWDOWN LIMIT: {mdd_pct}\nMARKET DATA: {market_analysis}\nCIO STRATEGY: {cio_strategy}\n\nINSTRUCTIONS: \n1. Open <thinking> tags. Review Sector Context and Barbell compliance.\n2. Close </thinking> tags. Issue a final veto or approval.\n\nREQUIRED FORMAT:\n<thinking>[Reasoning]</thinking>\nSector Exposure Risk: [Analysis]\nBarbell Compliance: [Check]\nMax Drawdown Estimate: [0% to 100%]\nVeto Status: [APPROVED/VETOED]\n\nCRITICAL: Do NOT use markdown.") | safe_llm | StrOutputParser()
        summary_runnable = PromptTemplate.from_template("SYSTEM: ROLE: Retail Financial Advisor.\nTARGET AUDIENCE: Retail investors.\nSTRATEGIES: {strategies}\nMONTE CARLO: {mc_projections}\nCONTEXT: {ticker_context}\nUSD/CAD RATE: {usdcad_rate}\nBUDGET: {budget_fmt}\nPRICES: {core_etfs_with_prices}\n\nINSTRUCTIONS: \n1. Open <thinking> tags. Compute exact shares (convert USD to CAD first). Calculate projected passive income.\n2. Close </thinking> tags. Output final action plan.\n\nREQUIRED FORMAT:\n<thinking>[Math]</thinking>\nPortfolio Stance: [BULLISH/BEARISH/NEUTRAL]\nAction Required: [BUY/SELL/HOLD]\nCapital Deployed: [0% to 100%]\nProjected Annual Income: [$ Value CAD]\nFinal Portfolio Breakdown: [e.g. XIU.TO: 60%, NVDA: 20%, Cash: 20%]\nExpected Return: [Projected Portfolio Value & ROI %]\nFinal Verdicts: [Summary]\nWealthsimple Trade Orders: [Step-by-step instructions with EXACT share counts]\nDisclaimer: [Warning]\n\nCRITICAL: Do NOT use markdown.") | safe_llm | StrOutputParser()

        reports = []; all_strategies = []; mc_list = []; ticker_context = []
        
        horizon_years = 1 if req.horizon == "short" else (10 if req.horizon == "long" else 5)
        core_weight = 1.0 - explore_frac
        explore_weight_per_ticker = explore_frac / len(raw_inputs)
        
        port_mu = core_weight * 0.06
        port_var = (core_weight ** 2) * (0.08 ** 2)

        tickers = []
        for r in raw_inputs:
            await push_progress(sid, "market", "running", r, f"Resolving database ticker...")
            t_res = await loop.run_in_executor(None, resolve_ticker_sync, r)
            if t_res: tickers.append(t_res)
        
        core_prices = await loop.run_in_executor(None, get_core_prices_sync)
        live_macro = await loop.run_in_executor(None, get_macro_data_cached_safe)
        
        try:
            macro_dict = json.loads(live_macro)
            usdcad_rate = macro_dict.get("USD_CAD", 1.35)
        except: usdcad_rate = 1.35
        
        for t in tickers:
            await push_progress(sid, "market", "running", t, "Analyzing Market Data & News...")
            await push_progress(sid, "macro", "running", t, "Classifying Macro Regime...")
            
            live_yf = await loop.run_in_executor(None, get_yf_data_safe, t, req.horizon)
            
            try:
                yf_dict = json.loads(live_yf)
                t_mu = float(yf_dict.get("Annual Expected Return (mu)", 0.08))
                t_sigma = float(yf_dict.get("Annual Volatility (sigma)", 0.15))
                if np.isnan(t_mu) or np.isnan(t_sigma):
                    t_mu, t_sigma = 0.08, 0.15
                
                t_mu = max(min(t_mu, 3.0), -1.0)
                t_sigma = max(min(t_sigma, 2.0), 0.01)
                    
                port_mu += explore_weight_per_ticker * t_mu
                port_var += (explore_weight_per_ticker ** 2) * (t_sigma ** 2)
                
                c_curr = yf_dict.get("Native Currency", "CAD")
                c_div = yf_dict.get("Dividend Yield", "0.00%")
                ticker_context.append(f"{t} -> Currency: {c_curr}, Dividend Yield: {c_div}")
            except Exception: pass
            
            res_market, res_macro = await asyncio.gather(
                safe_ainvoke(market_runnable, {"ticker": t, "yf_data": live_yf}),
                safe_ainvoke(macro_runnable, {"ticker": t, "horizon": req.horizon, "macro_data": live_macro})
            )
            
            await push_progress(sid, "market", "done", t, "Technical logic synthesized")
            await push_progress(sid, "macro", "done", t, "Macro environment classified")
            
            await push_progress(sid, "quant", "running", t, "De Prado statistical sizing...")
            res_quant = await safe_ainvoke(quant_runnable, {"ticker": t, "market_analysis": res_market, "macro_analysis": res_macro})
            await push_progress(sid, "quant", "done", t, "Alpha edge calculated")
            
            await push_progress(sid, "cio", "running", t, "Lhabitant portfolio architecture...")
            res_cio = await safe_ainvoke(cio_runnable, {**mandate, "ticker": t, "market_analysis": res_market, "macro_analysis": res_macro, "quant_analysis": res_quant})
            await push_progress(sid, "cio", "done", t, "Strategy drafted")
            
            await push_progress(sid, "risk", "running", t, "Taleb Sector Correlation test...")
            res_risk = await safe_ainvoke(risk_runnable, {"ticker": t, "cio_strategy": res_cio, "mdd_pct": mandate["mdd_pct"], "budget_fmt": mandate["budget_fmt"], "budget": str(mandate["budget"]), "market_analysis": res_market})
            await push_progress(sid, "risk", "done", t, "Asset audited")
            
            reports.append({"ticker": t, "market_analysis": res_market, "macro_analysis": res_macro, "quant_analysis": res_quant, "cio_strategy": res_cio, "risk_assessment": res_risk})
            all_strategies.append(res_cio)

        await push_progress(sid, "summary", "running", "Portfolio", "Calculating Forex & Expected Income...")
        
        summary_res = await safe_ainvoke(summary_runnable, {
            "tickers": ", ".join(tickers), "strategies": "\n".join(all_strategies),
            "budget_fmt": mandate["budget_fmt"], "core_etfs_with_prices": core_prices,
            "mc_projections": "\n".join(mc_list), "ticker_context": "\n".join(ticker_context),
            "usdcad_rate": str(usdcad_rate)
        })
        
        port_mu = max(min(port_mu, 3.0), -1.0)
        port_sigma = max(min(np.sqrt(port_var), 2.0), 0.01)
        mc_expected, mc_bull, mc_bear = [float(req.budget)], [float(req.budget)], [float(req.budget)]
        
        for y in range(1, horizon_years + 1):
            exp_val = req.budget * np.exp(port_mu * y)
            bull_val = req.budget * np.exp((port_mu - (port_sigma**2)/2) * y + port_sigma * 1.645 * np.sqrt(y))
            bear_val = req.budget * np.exp((port_mu - (port_sigma**2)/2) * y - port_sigma * 1.645 * np.sqrt(y))
            
            mc_expected.append(round(float(exp_val) if not np.isnan(exp_val) and not np.isinf(exp_val) else req.budget, 2))
            mc_bull.append(round(float(bull_val) if not np.isnan(bull_val) and not np.isinf(bull_val) else req.budget, 2))
            mc_bear.append(round(float(bear_val) if not np.isnan(bear_val) and not np.isinf(bear_val) else req.budget, 2))
            
        mc_graph_data = {
            "labels": [f"Year {y}" for y in range(horizon_years + 1)],
            "expected": mc_expected, "bull": mc_bull, "bear": mc_bear
        }

        await push_progress(sid, "summary", "done", "", "Orchestration Complete")
        await push_progress(sid, "pipeline", "complete")
        
        async def clear_q(): await asyncio.sleep(10); progress_queues.pop(sid, None)
        asyncio.create_task(clear_q())
        
        return {"mandate": mandate, "reports": reports, "executive_summary": summary_res, "mc_graph": mc_graph_data}

    except Exception as e:
        err_details = f"{type(e).__name__}: {str(e)}"
        traceback.print_exc()
        await push_progress(sid, "error", "error", detail=err_details)
        raise HTTPException(status_code=500, detail=err_details)

@app.post("/shutdown")
def shutdown():
    global bg_server
    if bg_server: bg_server.should_exit = True
    return {"message": "Server shutting down"}

html_content = """
<!DOCTYPE html>
<html lang="en" data-theme="dark">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Institutional AI Hedge Fund</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <link href="https://fonts.googleapis.com/css2?family=Cormorant+Garamond:ital,wght@0,300;0,400;0,600;1,400&family=Manrope:wght@300;400;500;700&display=swap" rel="stylesheet">
    <style>
        /* === THEME VARIABLES - DEEP GLASSMORPHISM === */
        :root, [data-theme="dark"] { 
            color-scheme: dark; --bg: #09090b; --text: #f0f0f0; --text-muted: #888; --line: rgba(255, 255, 255, 0.12); --nav-bg: rgba(9, 9, 11, 0.7);
            --glass-bg: rgba(20, 25, 35, 0.4); --glass-border: rgba(255, 255, 255, 0.08); --glass-shadow: 0 12px 40px 0 rgba(0, 0, 0, 0.5);
            --accent: #FF5A00; --accent-green: #34c759; --accent-red: #ff453a; --accent-amber: #ff9f0a; --badge-pill: rgba(0,0,0,0.4);
        }
        [data-theme="light"] { 
            color-scheme: light; --bg: #f4f5f7; --text: #111111; --text-muted: #666666; --line: rgba(0, 0, 0, 0.15); --nav-bg: rgba(244, 245, 247, 0.7);
            --glass-bg: rgba(255, 255, 255, 0.6); --glass-border: rgba(255, 255, 255, 0.9); --glass-shadow: 0 12px 40px 0 rgba(31, 38, 135, 0.08);
            --accent: #FF5A00; --accent-green: #28a745; --accent-red: #dc3545; --accent-amber: #f5a623; --badge-pill: rgba(255,255,255,0.7);
        }
        
        * { margin: 0; padding: 0; box-sizing: border-box; }
        html, body { max-width: 100vw; overflow-x: hidden; }
        body { 
            background-color: var(--bg); 
            background-image: radial-gradient(circle at 10% 20%, rgba(100, 210, 255, 0.07) 0%, transparent 40%), radial-gradient(circle at 90% 80%, rgba(255, 90, 0, 0.05) 0%, transparent 40%);
            background-attachment: fixed; color: var(--text); font-family: 'Manrope', sans-serif; transition: background-color 0.4s, color 0.4s; padding-top: 5rem; padding-bottom: 5rem; 
        }
        ::-webkit-scrollbar { width: 6px; height: 6px; }
        ::-webkit-scrollbar-track { background: transparent; }
        ::-webkit-scrollbar-thumb { background: var(--line); border-radius: 10px; }
        
        nav { position: fixed; top: 0; width: 100%; padding: 1rem 5%; background: var(--nav-bg); backdrop-filter: blur(16px); -webkit-backdrop-filter: blur(16px); border-bottom: 1px solid var(--line); z-index: 100; display: grid; grid-template-columns: 1fr auto 1fr; align-items: center; transition: 0.4s; }
        .btn-exit { justify-self: start; font-size: 1.5rem; background: none; border: none; color: var(--text-muted); cursor: pointer; transition: 0.3s; padding: 0; line-height: 1; }
        .btn-exit:hover { color: var(--accent-red); }
        .brand { justify-self: center; font-family: monospace; font-size: 0.8rem; letter-spacing: 0.15em; font-weight: 700; color: var(--text); text-transform: uppercase; }
        .btn-theme { justify-self: end; background: var(--glass-bg); border: 1px solid var(--line); padding: 0.5rem 1.2rem; border-radius: 20px; color: var(--text); cursor: pointer; font-size: 0.7rem; font-family: monospace; transition: 0.3s; text-transform: uppercase; letter-spacing: 0.05em; }
        .btn-theme:hover { border-color: var(--accent); color: var(--accent); }

        .container { width: 95%; max-width: 1000px; margin: 0 auto; box-sizing: border-box; }
        #form-container { max-width: 800px; margin: 0 auto 5rem auto; }
        #loader { display: none; flex-direction: column; justify-content: center; min-height: calc(100vh - 12rem); max-width: 800px; margin: 0 auto; }

        h1, h2, h3 { font-family: 'Cormorant Garamond', serif; font-weight: 400; color: var(--text); transition: 0.4s; }
        h1 { font-size: 4.5rem; line-height: 1; margin-bottom: 3rem; letter-spacing: -0.02em; }
        h2 { font-size: 2.5rem; margin-bottom: 1.5rem; }
        h3 { font-family: monospace; font-size: 0.8rem; text-transform: uppercase; letter-spacing: 0.15em; color: var(--accent); margin-bottom: 0.5rem; }
        
        .form-group { margin-bottom: 2.5rem; width: 100%; box-sizing: border-box; }
        label { display: block; font-family: monospace; font-size: 0.7rem; color: var(--text-muted); margin-bottom: 0.5rem; text-transform: uppercase; letter-spacing: 0.1em; }
        input { width: 100%; padding: 0.8rem 0; background: transparent !important; border: none; border-bottom: 1px solid var(--line); color: var(--text); font-size: 1.8rem; font-family: 'Cormorant Garamond', serif; outline: none; transition: 0.3s; border-radius: 0; box-sizing: border-box; }
        input:focus, input:hover { border-bottom-color: var(--accent); background: transparent !important; }
        input:-webkit-autofill { -webkit-transition: "color 9999s ease-out, background-color 9999s ease-out"; -webkit-transition-delay: 9999s; }
        input[type="number"]::-webkit-inner-spin-button, input[type="number"]::-webkit-outer-spin-button { -webkit-appearance: none; margin: 0; opacity: 0; }
        
        .btn-primary { display: inline-block; padding: 1.2rem 3rem; background: var(--glass-bg); backdrop-filter: blur(10px); color: var(--text); border: 1px solid var(--text); font-family: monospace; font-size: 0.85rem; text-transform: uppercase; letter-spacing: 0.1em; cursor: pointer; transition: 0.3s; margin-top: 1rem; border-radius: 4px; box-shadow: 0 4px 15px rgba(0,0,0,0.1); }
        .btn-primary:hover:not(:disabled) { background: var(--text); color: var(--bg); box-shadow: 0 8px 25px rgba(255, 90, 0, 0.3); border-color: transparent; }
        
        .agent-list { display: flex; flex-direction: column; gap: 0; border-top: 1px solid var(--line); margin-top: 1.5rem; }
        .agent-item { display: flex; justify-content: space-between; align-items: center; padding: 1.2rem 0; border-bottom: 1px solid var(--line); color: var(--text-muted); transition: 0.4s; font-family: monospace; text-transform: uppercase; font-size: 0.85rem; }
        .agent-item .left { display: flex; align-items: center; gap: 1rem; }
        .status-dot { width: 8px; height: 8px; border-radius: 50%; border: 1px solid var(--text-muted); transition: 0.4s; }
        .agent-item.running { color: var(--accent); }
        .agent-item.running .status-dot { background: var(--accent); border-color: var(--accent); box-shadow: 0 0 10px var(--accent); animation: pulse-dot 1.5s infinite; }
        .agent-item.done { color: var(--text); }
        .agent-item.done .status-dot { background: var(--accent-green); border-color: var(--accent-green); }
        @keyframes pulse-dot { 0%, 100% { opacity: 1; } 50% { opacity: 0.4; } }
        
        .progress-container { width: 100%; height: 2px; background: var(--line); margin-top: 2rem; border-radius: 2px; overflow: hidden; }
        .progress-bar { height: 100%; width: 0%; background: var(--accent); transition: width 0.4s ease-out; }

        #results { display: none; margin-top: 4rem; width: 100%; box-sizing: border-box; }
        
        .carousel-wrapper { position: relative; width: 100%; margin-bottom: 3rem; display: flex; align-items: center; box-sizing: border-box; }
        .carousel-track { display: flex; overflow-x: auto; scroll-snap-type: x mandatory; gap: 2rem; width: 100%; scrollbar-width: none; -ms-overflow-style: none; scroll-behavior: smooth; padding-bottom: 1rem; box-sizing: border-box; }
        .carousel-track::-webkit-scrollbar { display: none; }
        .carousel-slide { scroll-snap-align: center; min-width: 100%; width: 100%; flex-shrink: 0; margin-bottom: 0 !important; box-sizing: border-box; }
        
        .carousel-btn { position: absolute; top: 50%; transform: translateY(-50%); background: var(--glass-bg); backdrop-filter: blur(12px); border: 1px solid var(--line); color: var(--text); font-size: 1.5rem; width: 3rem; height: 3rem; border-radius: 50%; cursor: pointer; z-index: 10; display: flex; align-items: center; justify-content: center; transition: 0.3s; box-shadow: var(--glass-shadow); }
        .carousel-btn:hover { background: var(--accent); color: var(--bg); border-color: var(--accent); }
        .carousel-btn.left { left: -1.5rem; }
        .carousel-btn.right { right: -1.5rem; }
        @media (max-width: 768px) { .carousel-btn { display: none; } }

        .report-card { background: var(--glass-bg); backdrop-filter: blur(24px) saturate(150%); -webkit-backdrop-filter: blur(24px) saturate(150%); border: 1px solid var(--glass-border); border-radius: 12px; padding: 3rem; transition: border-color 0.4s, transform 0.4s, box-shadow 0.4s; box-shadow: var(--glass-shadow); width: 100%; box-sizing: border-box; }

        .glass-badge { display: flex; justify-content: space-between; align-items: center; width: 100%; flex-wrap: wrap; gap: 1rem; padding: 1.5rem 2rem; border-radius: 8px; backdrop-filter: blur(20px); -webkit-backdrop-filter: blur(20px); font-family: 'Manrope', sans-serif; font-weight: 700; font-size: 2.2rem; letter-spacing: 0.05em; margin-bottom: 2.5rem; border: 1px solid; box-shadow: 0 4px 20px rgba(0,0,0,0.2); box-sizing: border-box; }
        .glass-badge.buy { background: rgba(52,199,89,0.15); border-color: rgba(52,199,89,0.4); color: var(--accent-green); }
        .glass-badge.sell { background: rgba(255,69,58,0.15); border-color: rgba(255,69,58,0.4); color: var(--accent-red); }
        .glass-badge.hold { background: rgba(255,159,10,0.15); border-color: rgba(255,159,10,0.4); color: var(--accent-amber); }
        
        .badge-sub { font-family: monospace; font-size: 0.8rem; color: currentColor; text-transform: uppercase; font-weight: 700; letter-spacing: 0.1em; background: var(--badge-pill); padding: 0.4rem 0.8rem; border-radius: 4px; }

        .report-tabs { display: flex; border-bottom: 1px solid var(--line); margin-bottom: 2rem; overflow-x: auto; gap: 1.5rem; -webkit-overflow-scrolling: touch; }
        .report-tab { background: none; border: none; font-family: monospace; font-size: 0.75rem; color: var(--text-muted); text-transform: uppercase; padding: 0.8rem 0; cursor: pointer; border-bottom: 2px solid transparent; transition: 0.3s; white-space: nowrap; }
        .report-tab.active { color: var(--accent); border-bottom-color: var(--accent); }
        .report-tab-content { display: none; width: 100%; box-sizing: border-box; }
        .report-tab-content.active { display: block; }

        .data-table { width: 100%; border-collapse: collapse; margin-bottom: 2rem; table-layout: fixed; }
        .data-table td { padding: 1.2rem 1rem; border-bottom: 1px dotted var(--line); font-family: 'Cormorant Garamond', serif; font-size: 1.4rem; color: var(--text); word-wrap: break-word; overflow-wrap: break-word; }
        .data-table td:first-child { font-size: 0.75rem; color: var(--text-muted); font-family: monospace; text-transform: uppercase; width: 35%; letter-spacing: 0.05em; }
        
        .tag { display: inline-block; padding: 4px 10px; border-radius: 4px; font-family: monospace; font-size: 0.7rem; text-transform: uppercase; margin-left: 10px; vertical-align: middle; border: 1px solid; }
        .tag-green { color: var(--accent-green); border-color: rgba(52,199,89,0.3); background: rgba(52,199,89,0.05); }
        .tag-red { color: var(--accent-red); border-color: rgba(255,69,58,0.3); background: rgba(255,69,58,0.05); }
        .tag-amber { color: var(--accent-amber); border-color: rgba(255,159,10,0.3); background: rgba(255,159,10,0.05); }
        
        .prose-block { font-family: 'Cormorant Garamond', serif; font-size: 1.3rem; line-height: 1.8; color: var(--text); margin-bottom: 2rem; opacity: 0.95; }
        
        .ai-thought { margin-bottom: 2rem; border: 1px solid var(--glass-border); border-radius: 6px; background: rgba(0,0,0,0.15); backdrop-filter: blur(10px); transition: 0.3s; }
        [data-theme="light"] .ai-thought { background: rgba(255,255,255,0.3); }
        .ai-thought summary { font-family: monospace; font-size: 0.75rem; color: var(--accent); padding: 1rem; cursor: pointer; list-style: none; display: flex; align-items: center; text-transform: uppercase; letter-spacing: 0.1em; }
        .ai-thought summary::before { content: '▶'; margin-right: 10px; font-size: 0.6rem; transition: 0.2s; }
        .ai-thought[open] summary::before { transform: rotate(90deg); }
        .ai-thought-content { padding: 0 1rem 1rem; font-family: monospace; font-size: 0.8rem; color: var(--text-muted); white-space: pre-wrap; line-height: 1.6; border-top: 1px dotted var(--line); padding-top: 1rem; }
        
        .trade-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(260px, 1fr)); gap: 1.5rem; margin: 2rem 0; width: 100%; box-sizing: border-box; }
        .trade-card { background: var(--glass-bg); backdrop-filter: blur(16px); -webkit-backdrop-filter: blur(16px); border: 1px solid var(--glass-border); border-radius: 8px; padding: 1.5rem; display: flex; justify-content: space-between; align-items: center; transition: 0.3s; box-shadow: var(--glass-shadow); }
        .trade-card:hover { transform: translateY(-3px); box-shadow: 0 12px 30px rgba(0,0,0,0.4); border-color: var(--accent); }
        .tc-ticker { font-family: 'Manrope', sans-serif; font-weight: 700; font-size: 1.2rem; color: var(--text); }
        .tc-sub { font-family: monospace; font-size: 0.7rem; color: var(--text-muted); text-transform: uppercase; margin-top: 0.4rem; }
        .tc-val { font-family: 'Manrope', sans-serif; font-weight: 600; font-size: 1.2rem; color: var(--text); text-align: right; }
        
        canvas { background: var(--glass-bg); backdrop-filter: blur(12px); border-radius: 8px; padding: 1rem; border: 1px solid var(--glass-border); width: 100%; max-height: 400px; box-shadow: var(--glass-shadow); box-sizing: border-box; }

        @media (max-width: 768px) {
            .report-card { padding: 1.5rem; }
            .data-table td { font-size: 1.1rem; padding: 0.8rem 0.5rem; }
            .data-table td:first-child { width: 40%; font-size: 0.65rem; }
            h1 { font-size: 3rem; }
        }
    </style>
</head>
<body>
    <nav>
        <button type="button" class="btn-exit" onclick="shutdownServer()" title="Terminate">⏻</button>
        <div class="brand">HEDGE-FUND.AI</div>
        <button type="button" class="btn-theme" onclick="toggleTheme()" id="theme-label">LIGHT MODE</button>
    </nav>

    <div class="container">
        <div class="section" id="form-container">
            <h3>01 / Setup</h3>
            <h1>Capital Mandate.</h1>
            <form id="hedgeFundForm" onsubmit="return false;">
                <div class="form-group"><label>Total Capital (CAD)</label><input type="number" id="budget" value="5000"></div>
                <div class="form-group"><label>Temporal Horizon (Years)</label><input type="number" id="horizon" value="5"></div>
                <div class="form-group"><label>Max Drawdown (%)</label><input type="number" id="mdd" value="15"></div>
                <div class="form-group"><label>Target Assets (Comma separated)</label><input type="text" id="tickers" placeholder="e.g. Shopify, RY.TO, NVDA"></div>
                <button type="button" class="btn-primary" id="submitBtn" onclick="runPipeline()">Deploy Pipeline</button>
            </form>
        </div>

        <div class="section" id="loader">
            <div style="width: 100%;">
                <h3>02 / Processing</h3>
                <h2>Agent Orchestration.</h2>
                <div class="agent-list" id="agent-list"></div>
                <div class="progress-container"><div class="progress-bar" id="overall-progress"></div></div>
            </div>
        </div>

        <div id="results"></div>
    </div>

    <script>
        window.mcChartInstance = null; 
        const savedTheme = localStorage.getItem('theme') || 'dark';
        document.documentElement.setAttribute('data-theme', savedTheme);
        document.getElementById('theme-label').innerText = savedTheme === 'dark' ? 'LIGHT MODE' : 'DARK MODE';

        window.toggleTheme = function() {
            const isDark = document.documentElement.getAttribute('data-theme') === 'dark';
            const newTheme = isDark ? 'light' : 'dark';
            document.documentElement.setAttribute('data-theme', newTheme);
            localStorage.setItem('theme', newTheme);
            document.getElementById('theme-label').innerText = newTheme === 'dark' ? 'LIGHT MODE' : 'DARK MODE';
            
            if(window.mcChartInstance) {
                const gridColor = newTheme === 'dark' ? 'rgba(255,255,255,0.05)' : 'rgba(0,0,0,0.05)';
                const textColor = newTheme === 'dark' ? '#888' : '#666';
                window.mcChartInstance.options.scales.x.ticks.color = textColor;
                window.mcChartInstance.options.scales.x.grid.color = gridColor;
                window.mcChartInstance.options.scales.y.ticks.color = textColor;
                window.mcChartInstance.options.scales.y.grid.color = gridColor;
                window.mcChartInstance.options.plugins.legend.labels.color = textColor;
                window.mcChartInstance.update();
            }
        };

        window.scrollCarousel = function(direction) {
            const track = document.getElementById('asset-carousel');
            if (track) {
                const slideWidth = track.querySelector('.carousel-slide').offsetWidth;
                const maxScroll = track.scrollWidth - track.clientWidth;
                
                // Allow looping when clicking the arrows
                if (direction === 1 && Math.ceil(track.scrollLeft) >= maxScroll - 15) {
                    track.scrollTo({ left: 0, behavior: 'smooth' });
                } else if (direction === -1 && track.scrollLeft <= 15) {
                    track.scrollTo({ left: maxScroll, behavior: 'smooth' });
                } else {
                    track.scrollBy({ left: direction * (slideWidth + 32), behavior: 'smooth' }); 
                }
            }
        };

        window.escapeHTML = function(str) { 
            if (!str) return ''; 
            return str.replace(/[&<>'"]/g, tag => ({ '&': '&amp;', '<': '&lt;', '>': '&gt;', "'": '&#39;', '"': '&quot;' }[tag] || tag)); 
        };
        
        window.extractThinking = function(text) { 
            const regex = /<(?:thinking|think)>([\\s\\S]*?)<\\/(?:thinking|think)>/i; 
            const match = text ? text.match(regex) : null; 
            return match ? { thought: match[1].trim(), cleanText: text.replace(regex, '').trim() } : { thought: null, cleanText: text || "" }; 
        };
        
        window.extractKVPairs = function(text) { 
            if (!text) return []; 
            const pairs = [], regex = /^([A-Za-z][A-Za-z0-9\\s\\-&]+):\\s*(.+)$/gm; 
            let m; 
            while ((m = regex.exec(text)) !== null) { 
                if (m[1].length < 40) pairs.push({ key: m[1].trim(), value: m[2].trim() }); 
            } 
            return pairs; 
        };
        
        window.formatValue = function(val) { 
            return window.escapeHTML(val)
                .replace(/\\b(?:RISK.?ON|BULLISH|POSITIVELY|POSITIVE|STRONG|APPROVED|BUY)\\b/gi, '<span class=\"tag tag-green\">$&</span>')
                .replace(/\\b(?:RISK.?OFF|BEARISH|NEGATIVELY|NEGATIVE|WEAK|VETOED|AVOID|SELL)\\b/gi, '<span class=\"tag tag-red\">$&</span>')
                .replace(/\\b(?:NEUTRAL|TRANSITIONAL|DEFER|HOLD)\\b/gi, '<span class=\"tag tag-amber\">$&</span>'); 
        };
        
        window.formatProse = function(text) { 
            return window.escapeHTML(text)
                .replace(/^[-*]\\s+(.+)$/gm, '<span style=\"color:var(--accent); font-weight:bold; margin-right:8px;\">&bull;</span> $1')
                .replace(/(\\d+\\.)\\s/g, '<br><br><span style=\"color:var(--accent); font-weight:bold; font-family:monospace;\">$1</span> ')
                .replace(/\\n/g, '<br>'); 
        };
        
        window.detectVerdict = function(text) {
            if (!text) return { cls: 'hold', label: 'NEUTRAL' };
            const vetoMatch = text.match(/Veto\\s*Status:\\s*(APPROVED|VETOED)/i);
            const allocMatch = text.match(/Alpha\\s*Allocation:\\s*(BUY|AVOID|DEFER)/i);
            if (vetoMatch && vetoMatch[1].toUpperCase() === 'VETOED') return { cls: 'sell', label: 'WEAK' };
            if (allocMatch) {
                const v = allocMatch[1].toUpperCase();
                if (v === 'BUY') return { cls: 'buy', label: 'STRONG' };
                if (v === 'AVOID') return { cls: 'sell', label: 'WEAK' };
                if (v === 'DEFER') return { cls: 'hold', label: 'NEUTRAL' };
            }
            return { cls: 'hold', label: 'NEUTRAL' };
        };

        window.renderSection = function(rawText, isSummarizer = false, budgetAmt = null, allowChart = false) {
            const { thought, cleanText } = window.extractThinking(rawText);
            const kvPairs = window.extractKVPairs(cleanText);
            
            let html = `<div>`;
            if (thought) html += `<details class=\"ai-thought\"><summary>View AI Reasoning Chain</summary><div class=\"ai-thought-content\">${window.escapeHTML(thought)}</div></details>`;

            if (isSummarizer) {
                let breakdown = kvPairs.find(k => k.key.includes(\"Portfolio Breakdown\"));
                if (breakdown) {
                    const ws = []; 
                    const re = /([A-Z0-9.\\-]+)(?:\\s*:\\s*)(\\d+(?:\\.\\d+)?)%/gi; 
                    let m;
                    while ((m = re.exec(breakdown.value)) !== null) {
                        let dVal = budgetAmt ? ((parseFloat(m[2])/100)*budgetAmt).toLocaleString('en-US', {maximumFractionDigits:0}) : \"0\";
                        ws.push(`<div class=\"trade-card\"><div><div class=\"tc-ticker\">${m[1]}</div><div class=\"tc-sub\">Target Allocation</div></div><div><div class=\"tc-val\">$${dVal}</div><div class=\"tc-sub\" style=\"text-align:right;\">${m[2]}% WEIGHT</div></div></div>`);
                    }
                    if (ws.length > 0) html += `<h3>Actionable Trade Orders</h3><div class=\"trade-grid\">${ws.join('')}</div>`;
                }

                if (kvPairs.length > 0) {
                    html += `<table class=\"data-table\" style=\"margin-top: 2rem;\"><tbody>`;
                    kvPairs.forEach(kv => {
                        const kLow = kv.key.toLowerCase();
                        let skip = [\"portfolio breakdown\", \"wealthsimple trade orders\", \"disclaimer\", \"final verdicts\"].some(w => kLow.includes(w));
                        if (!skip) html += `<tr><td>${window.escapeHTML(kv.key)}</td><td>${window.formatValue(kv.value)}</td></tr>`;
                    });
                    html += `</tbody></table>`;
                }

                [\"Final Verdicts\", \"Brokerage Steps\", \"Disclaimer\"].forEach(tk => {
                    let match = kvPairs.find(k => k.key.includes(tk));
                    if (match) html += `<h3 style=\"margin-top:2.5rem; color:var(--accent);\">${tk}</h3><div class=\"prose-block\">${window.formatProse(match.value)}</div>`;
                });
                
            } else {
                if (kvPairs.length > 0) {
                    html += `<table class=\"data-table\"><tbody>`;
                    kvPairs.forEach(kv => {
                        const kLow = kv.key.toLowerCase();
                        let skip = [\"confidence\", \"drawdown\", \"score\", \"allocation target\", \"target weight\", \"conviction\", \"capital deployed\", \"portfolio risk\", \"construction\", \"breakdown\"].some(w => kLow.includes(w));
                        if (!skip) html += `<tr><td>${window.escapeHTML(kv.key)}</td><td>${window.formatValue(kv.value)}</td></tr>`;
                    });
                    html += `</tbody></table>`;
                } else {
                    html += `<div class=\"prose-block\">${window.formatProse(cleanText)}</div>`;
                }
            }
            html += `</div>`;
            return html;
        };

        window.runPipeline = async function() {
            const bVal = parseFloat(document.getElementById('budget').value) || 5000;
            const hVal = parseFloat(document.getElementById('horizon').value) || 5;
            const mVal = parseFloat(document.getElementById('mdd').value) || 15;
            const tVal = document.getElementById('tickers').value || \"XIU.TO\";
            let pHor = hVal < 2 ? \"short\" : hVal > 7 ? \"long\" : \"medium\";

            const btn = document.getElementById('submitBtn');
            btn.innerText = \"EXECUTING...\"; btn.disabled = true;
            
            document.getElementById('form-container').style.display = 'none';
            document.getElementById('loader').style.display = 'flex'; 
            
            const numTickers = tVal.split(',').filter(t => t.trim() !== '').length || 1;
            const totalSteps = (numTickers * 5) + 1; 
            let completedSteps = 0;
            document.getElementById('overall-progress').style.width = '0%';
            
            const sid = Math.random().toString(36).substring(7);
            const list = document.getElementById('agent-list');
            list.innerHTML = '';
            const agents = ['market', 'macro', 'quant', 'cio', 'risk', 'summary'];
            agents.forEach((a, i) => {
                list.innerHTML += `<div class=\"agent-item\" id=\"item-${a}\"><div class=\"left\"><div class=\"status-dot\"></div>0${i+1} / ${a}</div><span class=\"detail\" id=\"det-${a}\">PENDING</span></div>`;
            });
            
            const ev = new EventSource('/progress/' + sid);
            ev.onmessage = (e) => {
                try {
                    const d = JSON.parse(e.data);
                    const el = document.getElementById('item-' + d.agent);
                    const det = document.getElementById('det-' + d.agent);
                    
                    if(el && d.status !== 'error') {
                        el.className = 'agent-item ' + d.status;
                        if(d.detail) det.innerText = d.detail;
                        
                        if (d.status === 'done') {
                            completedSteps++;
                            let pct = Math.min(100, (completedSteps / totalSteps) * 100);
                            document.getElementById('overall-progress').style.width = pct + '%';
                        }
                    }
                } catch (err) { console.error(\"Event parse error:\", err); }
            };

            const payload = { session_id: sid, budget: bVal, horizon: pHor, mdd: mVal, tickers: tVal };

            try {
                const response = await fetch('/analyze', { method:'POST', headers:{'Content-Type':'application/json'}, body:JSON.stringify(payload) });
                if (!response.ok) {
                    const errText = await response.text();
                    let errMsg = errText;
                    try { errMsg = JSON.parse(errText).detail; } catch(e){}
                    throw new Error(errMsg);
                }
                const data = await response.json();
                window.renderResults(data, bVal);
            } catch (err) {
                alert(\"PIPELINE ERROR: \" + err.message);
                document.getElementById('loader').style.display = 'none';
                document.getElementById('form-container').style.display = 'block';
                btn.innerText = \"DEPLOY PIPELINE\"; btn.disabled = false;
            }
        };

        window.switchTab = function(groupId, tabId) {
            document.querySelectorAll(`.report-tab[onclick*=\"${groupId}\"]`).forEach(btn => btn.classList.toggle('active', btn.getAttribute('onclick').includes(tabId)));
            document.querySelectorAll(`[data-tabgroup=\"${groupId}\"].report-tab-content`).forEach(panel => panel.classList.toggle('active', panel.getAttribute('data-tabid') === tabId));
        };

        window.resetApp = function() {
            document.getElementById('results').style.display = 'none';
            document.getElementById('results').innerHTML = '';
            document.getElementById('form-container').style.display = 'block';
            
            const btn = document.getElementById('submitBtn');
            btn.innerText = \"DEPLOY PIPELINE\";
            btn.disabled = false;
            
            window.scrollTo({ top: 0, behavior: 'smooth' });
        };

        window.renderResults = function(data, budget) {
            document.getElementById('loader').style.display = 'none';
            const res = document.getElementById('results');
            let html = '';
            
            const hasMultiple = data.reports.length > 1;
            
            if (hasMultiple) {
                html += `<div class=\"carousel-wrapper\">\n                            <button type=\"button\" class=\"carousel-btn left\" onclick=\"scrollCarousel(-1)\">❮</button>\n                            <div class=\"carousel-track\" id=\"asset-carousel\">`;
            } else {
                html += `<div class=\"carousel-wrapper\"><div class=\"carousel-track single-track\" id=\"asset-carousel\">`;
            }
            
            data.reports.forEach((r, index) => {
                const verdict = window.detectVerdict(r.risk_assessment) || window.detectVerdict(r.cio_strategy) || window.detectVerdict(r.market_analysis);
                
                html += `<div class=\"report-card carousel-slide\">\n                            <h3>Asset Analysis</h3>\n                            \n                            <div class=\"glass-badge ${verdict.cls}\">\n                                <span>${window.escapeHTML(r.ticker)}</span>\n                                <span class=\"badge-sub\">${verdict.label}</span>\n                            </div>`;

                const tabId = 'tabs-' + index;
                const tabs = [
                    { id: 'market-'+index, label: '01 / Market', text: r.market_analysis },
                    { id: 'macro-'+index, label: '02 / Macro', text: r.macro_analysis },
                    { id: 'quant-'+index, label: '03 / Quant', text: r.quant_analysis },
                    { id: 'cio-'+index, label: '04 / CIO', text: r.cio_strategy },
                    { id: 'risk-'+index, label: '05 / Risk', text: r.risk_assessment },
                ];

                html += `<div class=\"report-tabs\" data-tabgroup=\"${tabId}\">`;
                tabs.forEach((tab, ti) => { html += `<button type=\"button\" class=\"report-tab${ti===0?' active':''}\" onclick=\"switchTab('${tabId}','${tab.id}')\">${tab.label}</button>`; });
                html += `</div>`;

                tabs.forEach((tab, ti) => {
                    html += `<div class=\"report-tab-content${ti===0?' active':''}\" data-tabgroup=\"${tabId}\" data-tabid=\"${tab.id}\">`;
                    html += window.renderSection(tab.text, false, budget, false);
                    html += `</div>`;
                });

                html += `</div>`; 
            });
            
            if (hasMultiple) {
                html += `   </div>\n                            <button type=\"button\" class=\"carousel-btn right\" onclick=\"scrollCarousel(1)\">❯</button>\n                         </div>`; 
            } else {
                html += `</div></div>`;
            }

            html += `<div class=\"report-card\">\n                        <h3>Final Output</h3>\n                        <h2 style=\"margin-bottom:1.5rem; border-bottom:1px solid var(--line); padding-bottom:1rem;\">Advisor Directives.</h2>`;
            
            html += window.renderSection(data.executive_summary, true, budget, false);
            
            if (data.mc_graph) {
                html += `<h3 style=\"margin-top:4rem;\">Portfolio Projection</h3>\n                         <h2 style=\"margin-bottom:1.5rem; border-bottom:1px solid var(--line); padding-bottom:1rem;\">Monte Carlo Simulation.</h2>\n                         <div style=\"width:100%;\"><canvas id=\"mcChart\"></canvas></div>`;
            }
            
            html += `</div>`;
            
            html += `<div id=\"resetBtnContainer\" style=\"text-align: center; margin: 4rem 0;\">\n                        <button type=\"button\" class=\"btn-primary\" style=\"padding: 1.5rem 4rem; font-size: 1rem;\" onclick=\"resetApp()\">RESEARCH OTHER STOCKS</button>\n                     </div>`;
            
            res.innerHTML = html;
            res.style.display = 'block';

            if (data.mc_graph) {
                setTimeout(() => {
                    const ctx = document.getElementById('mcChart').getContext('2d');
                    if (window.mcChartInstance) { window.mcChartInstance.destroy(); }

                    const isDark = document.documentElement.getAttribute('data-theme') === 'dark';
                    const gridColor = isDark ? 'rgba(255,255,255,0.05)' : 'rgba(0,0,0,0.05)';
                    const textColor = isDark ? '#888' : '#666';
                    
                    window.mcChartInstance = new Chart(ctx, {
                        type: 'line',
                        data: {
                            labels: data.mc_graph.labels,
                            datasets: [
                                { label: 'Bull Case (95%)', data: data.mc_graph.bull, borderColor: '#34c759', borderDash: [5, 5], fill: false, tension: 0.4 },
                                { label: 'Expected Trajectory', data: data.mc_graph.expected, borderColor: '#FF5A00', borderWidth: 3, fill: false, tension: 0.4 },
                                { label: 'Bear Case (5%)', data: data.mc_graph.bear, borderColor: '#ff453a', borderDash: [5, 5], fill: false, tension: 0.4 }
                            ]
                        },
                        options: {
                            responsive: true,
                            maintainAspectRatio: false, 
                            interaction: { mode: 'index', intersect: false },
                            plugins: { legend: { labels: { color: textColor, font: {family: 'monospace'} } } },
                            scales: {
                                x: { ticks: { color: textColor }, grid: { color: gridColor } },
                                y: { ticks: { color: textColor }, grid: { color: gridColor } }
                            }
                        }
                    });
                }, 100);
            }
        };

        window.shutdownServer = async function() {
            if(confirm("Terminate connection and shut down the background server?")) {
                try { await fetch('/shutdown', {method: 'POST'}); } catch(e) {}
                document.body.innerHTML = `<div style="display:flex; height:100vh; align-items:center; justify-content:center; background:var(--bg); color:var(--text);"><h1 style="color:var(--accent-red); border:none;">TERMINATED.</h1></div>`;
                setTimeout(() => window.close(), 3000);
            }
        };
    </script>
</body>
</html>
"""

@app.get("/")
async def serve_ui(): return HTMLResponse(content=html_content)

# === THE ULTIMATE FIX: UNIQUE PORT GENERATION ===
def get_free_port():
    while True:
        port = random.randint(8100, 8999)
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('127.0.0.1', port)) != 0:
                return port

PORT = get_free_port()

class CustomServer(uvicorn.Server):
    def install_signal_handlers(self):
        pass

config = uvicorn.Config(app, host="127.0.0.1", port=PORT, log_level="error")
bg_server = CustomServer(config=config)

def run_server():
    try: bg_server.run()
    except Exception: pass

threading.Thread(target=run_server, daemon=True).start()
time.sleep(1.5)

# Displays a minimalist, "Icy White" elegant deployment button
display(HTML(f"""
<style>
    .deploy-btn {{
        display: inline-block;
        padding: 20px 40px;
        /* Glassy White Base */
        background: rgba(255, 255, 255, 0.03);
        color: rgba(255, 255, 255, 0.9);
        text-decoration: none;
        font-family: 'Manrope', sans-serif;
        font-weight: 700;
        font-size: 18px;
        letter-spacing: 0.15em;
        border: 1px solid rgba(255, 255, 255, 0.2);
        border-radius: 4px;
        transition: all 0.5s cubic-bezier(0.2, 1, 0.3, 1);
        backdrop-filter: blur(15px);
        -webkit-backdrop-filter: blur(15px);
        text-transform: uppercase;
        cursor: pointer;
    }}
    
    .deploy-btn:hover {{
        /* Pure Glassy White Hover */
        background: rgba(255, 255, 255, 1);
        color: #09090b;
        border-color: #ffffff;
        transform: translateY(-4px);
        box-shadow: 0 20px 40px rgba(255, 255, 255, 0.15);
    }}

    .deploy-btn:active {{
        transform: translateY(-1px);
    }}

    .deploy-container {{
        text-align: center;
        padding: 50px;
        background: transparent;
    }}
</style>

<div class="deploy-container">
    <a href="http://127.0.0.1:{PORT}" target="_blank" class="deploy-btn">
        Launch Institutional Terminal
    </a>
    <p style="margin-top: 20px; font-family: monospace; font-size: 10px; color: #555; letter-spacing: 0.2em; text-transform: uppercase;">
        Terminal Ready • Secure Port {PORT}
    </p>
</div>
"""))